In [29]:
import pandas as pd
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

from hypex.dataset import Dataset, ExperimentData, InfoRole, TreatmentRole, TargetRole
from hypex.experiments.aa import AA_TEST, AA_TEST_WITH_STRATIFICATION
from hypex.splitters import AASplitter
from hypex.reporters.aa import AAPassedReporter, AABestSplitReporter

# Creation of a new test dataset with synthetic data. 

It is important to mark the data fields by assigning the appropriate roles:
- FeatureRole: a role for columns that contain features or predictor variables. Our split will be based on them. Applied by default if the role is not specified for the column.
- TreatmentRole: a role for columns that show the treatment or intervention.
- TargetRole: a role for columns that show the target or outcome variable.
- InfoRole: a role for columns that contain information about the data, such as user IDs. 

In [30]:
data = Dataset(
    roles={
        "user_id": InfoRole(int),
        "treat": TreatmentRole(int),
        "pre_spends": TargetRole(),
        "post_spends": TargetRole(),
        "gender": TargetRole()
    }, data="data.csv",
)
# data

# Cycled AA test
Then we create the test pipeline, experiment data, with which it will work and run it. We select one of the pre-assembled pipelines, in our case AA_TEST, or create a custom one. We can also adjust some of the preset parameters of the experiment by assigning them to the respective params of the experiment. I.e. here we set the range of the random states we want to run our AA test for. Then we run the experiment on our prepared dataset, wrapped into ExperimentData. 

In [31]:
aa = AA_TEST
aa.executors[0].params[AASplitter]= {"random_states": range(10)}
res = aa.execute(ExperimentData(data))
res

# Experiment results
To show the report with the summary of the test, we run the report method of the reporter, associated with the respective test type, AA test in our case.

AAPassedReporter shows the results of the number of tests (OK / NOT OK) for the different random states.

In [32]:
AAPassedReporter().report(res)

/home/tony_montana/job/HypEx/hypex/dataset/backends/pandas_backend.py:476: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return self.data.replace(to_replace=to_replace, value=value, regex=regex)


  Chi2Test aa test TTest aa test KSTest aa test TTest best split  \
0           NOT OK           NaN            NaN              NaN   
1              NaN        NOT OK         NOT OK               OK   
2              NaN            OK         NOT OK               OK   

  KSTest best split Chi2Test best split  result      feature group  
0               NaN                  OK  NOT OK       gender     0  
1                OK                 NaN  NOT OK  post_spends     0  
2                OK                 NaN      OK   pre_spends     0  

AABestSplitReporter returns the dataset with the best split among the ones covered by the cycled AA test.

In [33]:
AABestSplitReporter().report(res)

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0      0       488.0   414.444444   NaN      M   
1           1             8      1       512.5   462.222222  26.0    NaN   
2           2             7      1       483.0   479.444444  25.0      M   
3           3             0      0       501.5   424.333333  39.0      M   
4           4             1      1       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10      1       538.5   450.444444  42.0      M   
9996     9996             0      0       500.5   430.888889  26.0      F   
9997     9997             3      1       473.0   534.111111  22.0      F   
9998     9998             2      1       495.0   523.222222  67.0      F   
9999     9999             7      1       508.0   475.888889  38.0      F   

        industry    split  
0     E-commerce  control  
1     E-commerce     test  
2  

Then we repeat that for the AA test with stratification, also setting the share of the control group in the splits, we expect our test to generate.

In [34]:
aa = AA_TEST_WITH_STRATIFICATION
aa.executors[0].params[AASplitter]= {"random_states": range(10), "control_size": [0.3]}
res = aa.execute(ExperimentData(data))
res

In [35]:
AAPassedReporter().report(res)

/home/tony_montana/job/HypEx/hypex/dataset/backends/pandas_backend.py:476: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return self.data.replace(to_replace=to_replace, value=value, regex=regex)


  Chi2Test aa test TTest aa test KSTest aa test TTest best split  \
0           NOT OK           NaN            NaN              NaN   
1              NaN        NOT OK         NOT OK               OK   
2              NaN        NOT OK         NOT OK               OK   

  KSTest best split Chi2Test best split  result      feature group  
0               NaN                  OK  NOT OK       gender     0  
1                OK                 NaN  NOT OK  post_spends     0  
2                OK                 NaN  NOT OK   pre_spends     0  

In [36]:
AABestSplitReporter().report(res)

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0      0       488.0   414.444444   NaN      M   
1           1             8      1       512.5   462.222222  26.0    NaN   
2           2             7      1       483.0   479.444444  25.0      M   
3           3             0      0       501.5   424.333333  39.0      M   
4           4             1      1       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10      1       538.5   450.444444  42.0      M   
9996     9996             0      0       500.5   430.888889  26.0      F   
9997     9997             3      1       473.0   534.111111  22.0      F   
9998     9998             2      1       495.0   523.222222  67.0      F   
9999     9999             7      1       508.0   475.888889  38.0      F   

        industry    split  
0     E-commerce     test  
1     E-commerce  control  
2  